In [ ]:
!pip install tensorflow keras pillow numpy

In [ ]:
mkdir -p Data/{test/image,test/mask,train/image,train/mask}

In [ ]:
import os
import numpy as np
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.image import resize
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate

def build_unet(input_shape):
    inputs = Input(input_shape)

    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    # Decoder
    up3 = Conv2D(64, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv2))
    merge3 = concatenate([conv1, up3], axis=3)
    conv3 = Conv2D(64, 3, activation='relu', padding='same')(merge3)
    conv3 = Conv2D(64, 3, activation='relu', padding='same')(conv3)

    outputs = Conv2D(1, 1, activation='sigmoid')(conv3)

    model = Model(inputs=inputs, outputs=outputs)
    return model


def load_data(data_dir, mode="train"):
    image_dir = os.path.join(data_dir, mode, "image")
    mask_dir = os.path.join(data_dir, mode, "mask")

    images = []
    masks = []

    for filename in sorted(os.listdir(image_dir)):
        img = load_img(os.path.join(image_dir, filename), color_mode="grayscale")
        mask = load_img(os.path.join(mask_dir, filename), color_mode="grayscale")

        img = img_to_array(img) / 255.0
        mask = img_to_array(mask) / 255.0

        img = resize(img, [256, 256])
        mask = resize(mask, [256, 256])

        images.append(img)
        masks.append(mask)

    return np.array(images), np.array(masks)


# Parameters
input_shape = (256, 256, 1)
batch_size = 16
epochs = 50
learning_rate = 1e-4
data_dir = 'Data'

# Load data
x_train, y_train = load_data(data_dir, mode='train')
x_test, y_test = load_data(data_dir, mode='test')

# Build model
model = build_unet(input_shape)
model.compile(optimizer=Adam(learning_rate), loss=binary_crossentropy, metrics=['accuracy'])

# Train model
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test))

# Save model
model.save('retina_blood_vessel_segmentation_model2.h5')


Epoch 1/50
5/5 [==============================] - 5s 489ms/step - loss: 0.6834 - accuracy: 0.6794 - val_loss: 0.6643 - val_accuracy: 0.8384
Epoch 2/50
5/5 [==============================] - 2s 443ms/step - loss: 0.6487 - accuracy: 0.8387 - val_loss: 0.6241 - val_accuracy: 0.8384
Epoch 3/50
5/5 [==============================] - 2s 442ms/step - loss: 0.6034 - accuracy: 0.8387 - val_loss: 0.5698 - val_accuracy: 0.8384
Epoch 4/50
5/5 [==============================] - 2s 446ms/step - loss: 0.5416 - accuracy: 0.8387 - val_loss: 0.4988 - val_accuracy: 0.8384
Epoch 5/50
5/5 [==============================] - 2s 451ms/step - loss: 0.4703 - accuracy: 0.8387 - val_loss: 0.4431 - val_accuracy: 0.8384
Epoch 6/50
5/5 [==============================] - 2s 452ms/step - loss: 0.4432 - accuracy: 0.8387 - val_loss: 0.4535 - val_accuracy: 0.8384
Epoch 7/50
5/5 [==============================] - 2s 450ms/step - loss: 0.4471 - accuracy: 0.8387 - val_loss: 0.4389 - val_accuracy: 0.8384
Epoch 8/50
5/5 [====

In [ ]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array, load_img

# Define paths
original_images_path = 'Data/train/image'
manual_segmented_images_path = 'Data/train/mask'

# Get list of image files
original_images_files = os.listdir(original_images_path)
manual_segmented_images_files = os.listdir(manual_segmented_images_path)

# Initialize lists to store images
original_images = []
manual_segmented_images = []

# Load original images
for image_file in original_images_files:
    image = load_img(os.path.join(original_images_path, image_file), color_mode='grayscale')
    image = img_to_array(image)
    original_images.append(image)

# Load manual segmented images
for image_file in manual_segmented_images_files:
    image = load_img(os.path.join(manual_segmented_images_path, image_file), color_mode='grayscale')
    image = img_to_array(image)
    manual_segmented_images.append(image)

# Convert lists to numpy arrays
original_images = np.array(original_images, dtype='float') / 255
manual_segmented_images = np.array(manual_segmented_images, dtype='float') / 255

# Define the model
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(None, None, 1)))
model.add(MaxPooling2D((2, 2), padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(1, (3, 3), activation='sigmoid', padding='same'))

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(original_images, manual_segmented_images, epochs=50, batch_size=2, shuffle=True)

model.save('retina_model.h5')


Epoch 1/50
40/40 [==============================] - 10s 92ms/step - loss: 0.3318 - accuracy: 0.8561
Epoch 2/50
40/40 [==============================] - 4s 91ms/step - loss: 0.2657 - accuracy: 0.8768
Epoch 3/50
40/40 [==============================] - 4s 92ms/step - loss: 0.2649 - accuracy: 0.8768
Epoch 4/50
40/40 [==============================] - 4s 93ms/step - loss: 0.2628 - accuracy: 0.8768
Epoch 5/50
40/40 [==============================] - 4s 93ms/step - loss: 0.2463 - accuracy: 0.8768
Epoch 6/50
40/40 [==============================] - 4s 93ms/step - loss: 0.2142 - accuracy: 0.8807
Epoch 7/50
40/40 [==============================] - 4s 94ms/step - loss: 0.1878 - accuracy: 0.8960
Epoch 8/50
40/40 [==============================] - 4s 94ms/step - loss: 0.1753 - accuracy: 0.8993
Epoch 9/50
40/40 [==============================] - 4s 94ms/step - loss: 0.1601 - accuracy: 0.9033
Epoch 10/50
40/40 [==============================] - 4s 94ms/step - loss: 0.1515 - accuracy: 0.9051
Epoch 11